In [1]:
import numpy as np
import xarray as xr
import scipy
import gdal
from scipy.ndimage.filters import uniform_filter
from scipy.ndimage.filters import *
import os


import matplotlib.pyplot as plt
#import matplotlib.image as mpimg
%matplotlib inline

In [2]:
S1 = np.array([[ 0,  1,  2, 3, 4],
                [5,  6,  7, 8, 9],
                [10, 11, 12, 13, 14],
                [15, 15, 17, 18, 19],
                [20, 21, 22, 23, 24]],dtype = np.float32)

In [3]:
#Create a circular search window.

radius = 2

y, x = np.ogrid[-radius: radius + 1, -radius: radius + 1]
shape = x**2 + y**2 <= radius**2
shape


array([[False, False,  True, False, False],
       [False,  True,  True,  True, False],
       [ True,  True,  True,  True,  True],
       [False,  True,  True,  True, False],
       [False, False,  True, False, False]], dtype=bool)

In [4]:
#This just sums the number of values collected by the circular window so that it can be used 
#in the calculation of percentile.

denominator = sum(sum(shape > 0))
denominator

#Note: using median like this only gives the correct value for circles with odd radius values.

13

In [5]:
#Setup a 1d array based on the size of the cicular window. This is simply to find the central value of the range for
#the percentile calculation.

value_range = np.arange(0,denominator + 1)

central_value = int(np.median(value_range))
print(central_value)


6


In [6]:
#This function calculates the percentile of a cebtral value within its circular neighborhood.

def function(x):
    centroid = x[central_value] #Find the centre of the circle and read the value.
    y = sum(x > centroid)/denominator #Count number of values greater than centroid value
    return y

function(S1)

IndexError: index 6 is out of bounds for axis 0 with size 5

In [7]:
test = generic_filter(S1, function, footprint= shape, mode='constant')

In [8]:
test

array([[ 0.38461539,  0.46153846,  0.46153846,  0.38461539,  0.23076923],
       [ 0.38461539,  0.46153846,  0.46153846,  0.38461539,  0.23076923],
       [ 0.38461539,  0.46153846,  0.46153846,  0.38461539,  0.23076923],
       [ 0.23076923,  0.38461539,  0.38461539,  0.30769232,  0.15384616],
       [ 0.15384616,  0.15384616,  0.15384616,  0.07692308,  0.        ]], dtype=float32)